# KMeans-Clustering

Algorithmus und *Scikit-learn*-Estimator

Datensatz: *Old Faithful Geyser*, aus *R* exportiert

In [ ]:
import pandas as pd
import numpy as np
import matplotlib # for colormap
import matplotlib.pyplot as plt
from sklearn import compose, preprocessing

Einlesen und Analyse der Daten

In [ ]:
faithful = pd.read_csv('faithful.csv')
print(faithful.info())
faithful.describe()

In [ ]:
faithful.head()

Visuelle Darstellung

- 2 Häufungen erkennbar
- Wertebereiche x, y verschieden

In [ ]:
_ = faithful.plot.scatter('waiting', 'eruptions')

Skalierung der Daten

In [ ]:
prepr = compose.ColumnTransformer([
    ('num', preprocessing.StandardScaler(), ['waiting', 'eruptions']),
])
X = prepr.fit_transform(faithful)
X.shape

Kontrollplot

In [ ]:
plt.scatter(X[:,0], X[:,1])
plt.show()

Festlegung Cluster-Zahl

In [ ]:
K = 2

Zufällige Festlegung der Cluster-Zentren

In [ ]:
centers = np.random.uniform(-1, 1, (K, 2))
centers

Hilfsfunktion: Plot der klassifizierten Daten und Clusterzentren

In [ ]:
def plot_knn(X, clusters, centers):
    cmap = matplotlib.colormaps['jet']
    colors = cmap(np.linspace(0, 1, centers.shape[0] + 1)) # K + 1 Farben
    cl_colors = [colors[c] for c in clusters]
    cent_colors = colors[0:-1] # K Werte
    plt.scatter(X[:,0], X[:, 1], c = cl_colors, alpha = 0.5, s=20)
    plt.scatter(centers[:,0], centers[:,1], c = cent_colors, marker='*', s=200)
    #plt.show()

Test, noch keine Cluster zugeordnet (Dummy-Cluster K=2)

In [ ]:
plot_knn(X, np.full(X.shape[0], K), centers)

Wiederhole:

- Bestimme Abstände zu den Clusterzentren und wähle Zentrum-Nummer kleinsten Abstand
- Verschiebe Clusterzentren in die Mitte der Cluster

Bestimmen der Abstände aller Datenpunkte zu allen Cluster-Zentren

In [ ]:
def get_dists(X, centers):
    K = centers.shape[0]
    dists = list()
    for k in range(K):
        d = np.square(X - centers[k,:].reshape(1, 2))
        d = np.sum(d, axis=1)
        #print(d.shape)
        dists.append(d.reshape(-1,1))
    return np.hstack(dists)

Auswahl des Clusters mit dem kleinsten Abstand zum Zentrum

In [ ]:
def get_cluster(X, centers):
    K = centers.shape[0]
    d = get_dists(X, centers)
    return np.argmin(d, axis=1)

Test der Funktionen

In [ ]:
d = get_dists(X, centers)
print(d[:5,:])
clusters = get_cluster(X, centers)
print(clusters[:5])

Plot mit den ermittelten Clustern

In [ ]:
plot_knn(X, clusters, centers)

Neuzentrieren der Cluster-Zentren: Auswahl des Schwerpunkts

In [ ]:
def recenter(X, cluster, K):
    centers = list()
    for k in range(K):
        idx = np.where(cluster==k)
        #print(idx)
        if len(idx):
            c = np.mean(X[clusters==k,:], axis=0)
        else:
            c = (np.inf, np.inf)
        centers.append(c)
    return np.vstack(centers)

In [ ]:
centers = recenter(X, clusters, K)

Plot der neu festgelegten Cluster-Zentren

In [ ]:
plot_knn(X, clusters, centers)

… und wiederhole beide Schritte bis zur gewünschten Genauigkeit

Illustration 5 Iterationen

In [ ]:
K = 3
iterations = 5
centers = np.random.uniform(-1, 1, (K, 2))
# dummy Cluster K
clusters = np.full(X.shape[0], K)
nr = 0
plt.figure(figsize=(10, 20))
for i in range(iterations):
    nr += 1
    plt.subplot(iterations, 2, nr)
    plot_knn(X, clusters, centers)
    plt.title('Recenter {}'.format(i+1))
    clusters = get_cluster(X, centers)
    nr += 1
    plt.subplot(iterations, 2, nr)
    plot_knn(X, clusters, centers)
    plt.title('Get cluster {}'.format(i+1))
    centers = recenter(X, clusters, K)

## KMeans-Clustering in Scikit-Learn

In [ ]:
from sklearn.cluster import KMeans

Initialisieren und Trainieren, `n_init` legt die Zahl der Versuche für initiale Auswahl der Cluster-Zentren fest.

In [ ]:
kmean = KMeans(n_init='auto', n_clusters=3)
kmean.fit(X)

Darstellung des Cluster.

In [ ]:
plot_knn(X, kmean.labels_, kmean.cluster_centers_)